# People Flow Detection using Object Tracking Visualization

---

## 📋 Overview
This project implements a sophisticated system to detect and count people entering and exiting a defined area in a video using object tracking and heatmap visualization. It leverages **YOLOv8** for detection and **ByteTrack** for tracking, complemented by a heatmap to illustrate movement intensity. Bounding boxes are drawn in green with unique track IDs for clear identification.

---

## 🔍 Detection Method
- **Model**: YOLOv8 (pre-trained `yolov8n.pt`) for detecting persons (class 0).
- **Tracker**: ByteTrack ensures consistent tracking of individuals across frames.
- **Libraries**: Utilizes OpenCV, Ultralytics YOLO, Supervision, and Matplotlib for processing and visualization.

---

## 📏 Line Coordinates
- **Upper Line (IN)**: Positioned at `y = frame height / 3`, marked in **blue**.
- **Lower Line (OUT)**: Positioned at `y = 2 * frame height / 3`, marked in **red**.
- Defined using the [PolygonZone](https://polygonzone.roboflow.com/) framework.

---

## 🧮 Logic for IN/OUT Counting
- **IN**: A person is counted as entering when their trajectory moves downward, crossing the upper line (y-coordinate transitions from below `LINE_Y1` to at or above `LINE_Y1`).
- **OUT**: A person is counted as exiting when their trajectory moves upward, crossing the lower line (y-coordinate transitions from above `LINE_Y2` to at or below `LINE_Y2`).
- The system tracks the center y-coordinate of bounding boxes to determine direction, updating counters based on these crossings.

---

## 📤 Output
- **Video**: `output_video.mp4` displays bounding boxes, track IDs, color-coded lines, and live IN/OUT counters.
- **Heatmap**: `heatmap.png` visualizes movement intensity using bounding box center points, normalized with a 'hot' colormap.

---

## 📥 Input & Output Files
- **Input Video**: Paste your video file path here (e.g., `/content/people-walking video.mp4`).
- **Output People Flow Detection Video**: Save as `output_video.mp4`.
- **Heatmap Image**: Save as `heatmap.png`.

---

## 🚀 Setup
1. Install required dependencies in your Colab environment:
   ```bash
   !pip install opencv-python numpy ultralytics supervision matplotlib


# Import Libraries

In [34]:
import cv2
import numpy as np
from ultralytics import YOLO
from supervision import ByteTrack, Color
import supervision as sv
import matplotlib.pyplot as plt

# Load YOLOv8n model

In [35]:
model = YOLO("yolov8n.pt")

# Video input


In [36]:
video_path = "/content/people-walking video.mp4"
cap = cv2.VideoCapture(video_path)


# Get video properties


In [37]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define two horizontal lines


In [38]:
LINE_Y1 = height // 3  # Upper line
LINE_Y2 = 2 * height // 3  # Lower line
LINE_COLOR_UP = Color.BLUE
LINE_COLOR_DOWN = Color.RED

# Initialize counters and tracker


In [39]:
in_count = 0
out_count = 0
tracker = ByteTrack()
track_history = {}
heatmap = np.zeros((height, width), dtype=np.float32)

# Video writer for output


In [40]:
output_path = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Line coordinates for counting


In [41]:
line1 = sv.LineZone(start=sv.Point(0, LINE_Y1), end=sv.Point(width, LINE_Y1))
line2 = sv.LineZone(start=sv.Point(0, LINE_Y2), end=sv.Point(width, LINE_Y2))

def process_frame(frame, frame_num):
    global in_count, out_count

    # Run YOLO detection
    results = model(frame, classes=[0])[0]  # Class 0 for person
    detections = sv.Detections.from_ultralytics(results)

    # Update tracker
    detections = tracker.update_with_detections(detections)

    # Draw bounding boxes and IDs
    annotated_frame = frame.copy()
    for box, track_id in zip(detections.xyxy, detections.tracker_id):
        x1, y1, x2, y2 = map(int, box)
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        # Update track history with (x, y) coordinates
        if track_id not in track_history:
            track_history[track_id] = []
        track_history[track_id].append((center_x, center_y))

        # Draw bounding box and ID
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(annotated_frame, f"ID: {track_id}", (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Count IN/OUT
        if len(track_history[track_id]) > 1:
            prev_x, prev_y = track_history[track_id][-2]
            curr_x, curr_y = track_history[track_id][-1]

            # IN: Moving down, crossing upper line
            if prev_y < LINE_Y1 and curr_y >= LINE_Y1:
                in_count += 1
            # OUT: Moving up, crossing lower line
            elif prev_y > LINE_Y2 and curr_y <= LINE_Y2:
                out_count += 1
     # Draw lines
    cv2.line(annotated_frame, (0, LINE_Y1), (width, LINE_Y1), LINE_COLOR_UP.as_bgr(), 2)
    cv2.line(annotated_frame, (0, LINE_Y2), (width, LINE_Y2), LINE_COLOR_DOWN.as_bgr(), 2)

    # Display counters
    cv2.putText(annotated_frame, f"IN: {in_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(annotated_frame, f"OUT: {out_count}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    return annotated_frame

# Process video


In [42]:
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    annotated_frame = process_frame(frame, frame_num)
    out.write(annotated_frame)
    frame_num += 1


0: 384x640 37 persons, 95.4ms
Speed: 3.6ms preprocess, 95.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 108.7ms
Speed: 2.7ms preprocess, 108.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 35 persons, 106.4ms
Speed: 2.2ms preprocess, 106.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 persons, 108.9ms
Speed: 2.8ms preprocess, 108.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 persons, 95.6ms
Speed: 3.0ms preprocess, 95.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 96.5ms
Speed: 2.2ms preprocess, 96.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons, 97.8ms
Speed: 2.2ms preprocess, 97.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons, 96.4ms
Speed: 2.2ms preprocess, 96.4ms inference, 1.0ms postprocess per im

# Release video resources


In [48]:
cap.release()
out.release()


# Generate & save improved heatmap


In [49]:
heatmap = np.zeros((height, width), dtype=np.float32)
for track_id in track_history:
    for center_x, center_y in track_history[track_id]:
        if 0 <= center_y < height and 0 <= center_x < width:
            heatmap[int(center_y), int(center_x)] += 1

# Gaussian blur for smoothing


In [50]:
heatmap = cv2.GaussianBlur(heatmap, (21, 21), 10)

# Normalize and clip the heatmap
heatmap = np.clip(heatmap, 0, None)
if heatmap.max() > 0:
    heatmap = heatmap / heatmap.max()

# Visualize and save heatmap


In [51]:
plt.figure(figsize=(10, 6))
plt.imshow(heatmap, cmap='hot', interpolation='bilinear', vmin=0, vmax=0.8)
plt.colorbar(label='Intensity')
plt.title('People Movement Heatmap')
plt.axis('off')  # Hide axes for cleaner look
plt.savefig('heatmap.png', bbox_inches='tight', pad_inches=0)
plt.close()

In [ ]:
## 📥 Input & Output Files

### 🎥 Input Video:
[![Input Video SS](https://github.com/Moynuddin780/People-Flow-Detection-using-Object-Tracking/blob/main/Input%20video%20SS.jpg)

### 🎥 Output Video:
[![Output Video SS](https://github.com/Moynuddin780/People-Flow-Detection-using-Object-Tracking/blob/main/Output%20ScreenShoot.jpg)